<a href="https://colab.research.google.com/github/dbtjr1103/Basic_practice/blob/main/LDA_RAW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

In [2]:
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

env: JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"


In [ ]:
%%bash
bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
pip3 install /tmp/mecab-python-0.996

In [22]:
import pandas as pd

from konlpy.tag import Mecab

import time
import numpy as np

from gensim import corpora
import gensim

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
df = pd.read_csv('/content/data_csv.csv', encoding='utf-8-sig')

In [23]:
df.head()

,comment,author,date,num_likes,label
0,"되도안한 뉴스 알고리즘에 돈, 시간, 인력 쓰지 말고 지하철 시스템과 연동해 물류를...",D. Light Edward,2021-12-28T10:41:52Z,757,0
1,지하철 물류배송은 굉장히 좋은 생각 같습니다.<br>외곽과 연결도 이미 되어있고<b...,Union DNA,2021-12-31T16:23:20Z,510,0
2,공간이 부를 창출하고 계급을 벗어나게 한다.. 앞선 영상부터 빌드업이 엄청나네요. ...,S.R Lee,2021-12-28T15:26:46Z,458,0
3,와아...건축학이 단순 건축물에 관한 것인줄로만 알았는데 교수님 강의(?) 들으면 ...,Eswaii Son,2021-12-28T11:58:02Z,336,0
4,얼마나 많은 공부와 고민을 하셨을지..<br>교수님과 같이 미래를 창조하려고 노력하...,ShineBell,2021-12-28T10:50:41Z,330,0


In [30]:
comment = df['comment'].tolist()
comment

['되도안한 뉴스 알고리즘에 돈, 시간, 인력 쓰지 말고 지하철 시스템과 연동해 물류를 옮길 알고리즘을 개발하는게 나라를 위해서도 도움이 되는일 아닐까? 지금까지 듣던중 가장 신박한 제안이였습니다. 교수님~! 서울시나 국토부에서 이 영상을 보고 진지하게 논의 되면 좋겠습니다. 근데 이게 실현이 된다면... 물류, 택배, 배송 업계의 극심한 반대가 예상됩니다. 대량실직이니 뭐니 하면서~',
 '지하철 물류배송은 굉장히 좋은 생각 같습니다.<br>외곽과 연결도 이미 되어있고<br>운영 안하는 야간을 이용하면 굉장히 많은 양을 처리할 수 있을 것 같네요.',
 '공간이 부를 창출하고 계급을 벗어나게 한다.. 앞선 영상부터 빌드업이 엄청나네요. 아이디어가 너무 좋습니다. 특히나 우리나라는 유통 산업에서 불필요한 지출이 많이 나는 구조라 더더욱 이러한 개선이 필요하다고 봅니다. 나머지 기술적으로 부족해보이는 부분은 다른 토목 전문가가 해결할 수 있을거예요!ㅎㅎ 그리고 저도 쥘 베른의 선구안을 정말 존경합니다. 저도 쥘 베른이나 유 교수님처럼 지식인이 되고 싶다는 생각이 강하게 드네요.',
 '와아...건축학이 단순 건축물에 관한 것인줄로만 알았는데 교수님 강의(?) 들으면 이건 사회학, 인문철학, 역사까지 아우러는데요!!! 많이 배워갑니다. 그리고 교수님의 제안, 진짜 이뤄졌으면 좋겠습니다. 우리나라면 할 수 있을 것 같아요!!!',
 '얼마나 많은 공부와 고민을 하셨을지..<br>교수님과 같이 미래를 창조하려고 노력하는 어른이 더 많아졌으면.. 하면서도 저도 그런 어른이 되고 싶네요.<br>오늘도 엄청난 영감 주셔서 감사합니다.',
 '드론을 통한 자율주행 성능 개선 관련 연구하는 사람입니다. 드론의 고질적인 적은 payload, 이를 보완하기 위한 무거운 배터리의 악순환, 어마어마한 소음 문제를 정말 잘 꿰뚫어보셨네요. 너무나 공감합니다. 말씀하신 인프라가 한국에 제일 먼저 도입될지는 아직 미지수지만 만약 된다면 다행히 자율주행 로봇 쪽으로 먹고 살 길은 있겠군요.

In [36]:
comment[0]

'되도안한 뉴스 알고리즘에 돈, 시간, 인력 쓰지 말고 지하철 시스템과 연동해 물류를 옮길 알고리즘을 개발하는게 나라를 위해서도 도움이 되는일 아닐까? 지금까지 듣던중 가장 신박한 제안이였습니다. 교수님~! 서울시나 국토부에서 이 영상을 보고 진지하게 논의 되면 좋겠습니다. 근데 이게 실현이 된다면... 물류, 택배, 배송 업계의 극심한 반대가 예상됩니다. 대량실직이니 뭐니 하면서~'

In [57]:
import re
for i in comment:
    text = re.sub('[^\s 가-힣 A-Z a-z 0-9]|(\w+:\/\/\/S+)|(a href)', '', i).replace("br","")
    print(text)

<>:3: DeprecationWarning: invalid escape sequence \s
<>:3: DeprecationWarning: invalid escape sequence \s
<ipython-input-57-de9603e10eeb>:3: DeprecationWarning: invalid escape sequence \s
  text = re.sub('[^\s 가-힣 A-Z a-z 0-9]|(\w+:\/\/\/S+)|(a href)', '', i).replace("br","")


되도안한 뉴스 알고리즘에 돈 시간 인력 쓰지 말고 지하철 시스템과 연동해 물류를 옮길 알고리즘을 개발하는게 나라를 위해서도 도움이 되는일 아닐까 지금까지 듣던중 가장 신박한 제안이였습니다 교수님 서울시나 국토부에서 이 영상을 보고 진지하게 논의 되면 좋겠습니다 근데 이게 실현이 된다면 물류 택배 배송 업계의 극심한 반대가 예상됩니다 대량실직이니 뭐니 하면서
지하철 물류배송은 굉장히 좋은 생각 같습니다외곽과 연결도 이미 되어있고운영 안하는 야간을 이용하면 굉장히 많은 양을 처리할 수 있을 것 같네요
공간이 부를 창출하고 계급을 벗어나게 한다 앞선 영상부터 빌드업이 엄청나네요 아이디어가 너무 좋습니다 특히나 우리나라는 유통 산업에서 불필요한 지출이 많이 나는 구조라 더더욱 이러한 개선이 필요하다고 봅니다 나머지 기술적으로 부족해보이는 부분은 다른 토목 전문가가 해결할 수 있을거예요 그리고 저도 쥘 베른의 선구안을 정말 존경합니다 저도 쥘 베른이나 유 교수님처럼 지식인이 되고 싶다는 생각이 강하게 드네요
와아건축학이 단순 건축물에 관한 것인줄로만 알았는데 교수님 강의 들으면 이건 사회학 인문철학 역사까지 아우러는데요 많이 배워갑니다 그리고 교수님의 제안 진짜 이뤄졌으면 좋겠습니다 우리나라면 할 수 있을 것 같아요
얼마나 많은 공부와 고민을 하셨을지교수님과 같이 미래를 창조하려고 노력하는 어른이 더 많아졌으면 하면서도 저도 그런 어른이 되고 싶네요오늘도 엄청난 영감 주셔서 감사합니다
드론을 통한 자율주행 성능 개선 관련 연구하는 사람입니다 드론의 고질적인 적은 payload 이를 보완하기 위한 무거운 배터리의 악순환 어마어마한 소음 문제를 정말 잘 꿰뚫어보셨네요 너무나 공감합니다 말씀하신 인프라가 한국에 제일 먼저 도입될지는 아직 미지수지만 만약 된다면 다행히 자율주행 로봇 쪽으로 먹고 살 길은 있겠군요 좋은 아이디어 공유 감사합니다
단순히 공원을 만들기위해서 시작 된 고민들일 텐데 이렇게까지 유기적인 사고의 흐름으로 이어진다는 게 대단한 것 같습니다 도시는 무엇으로

In [41]:
m = Mecab()

def tokenizer(text):
    return m.pos(text)

In [42]:
okt = Okt()

In [43]:
len(comment)

1624

In [44]:
tokenized_doc = []
for i in range(len(df)):
    tmp = tokenizer(df.iloc[i, 0])
    tokenized_doc.append(tmp)

In [10]:
print(tokenized_doc[0])

[('되', 'VV'), ('도', 'EC'), ('안', 'MAG'), ('한', 'VV+ETM'), ('뉴스', 'NNG'), ('알고리즘', 'NNG'), ('에', 'JKB'), ('돈', 'NNG'), (',', 'SC'), ('시간', 'NNG'), (',', 'SC'), ('인력', 'NNG'), ('쓰', 'VV'), ('지', 'EC'), ('말', 'VX'), ('고', 'EC'), ('지하철', 'NNG'), ('시스템', 'NNG'), ('과', 'JC'), ('연동', 'NNG'), ('해', 'XSV+EC'), ('물류', 'NNP'), ('를', 'JKO'), ('옮길', 'VV+ETM'), ('알고리즘', 'NNG'), ('을', 'JKO'), ('개발', 'NNG'), ('하', 'XSV'), ('는', 'ETM'), ('게', 'NNB+JKS'), ('나라', 'NNG'), ('를', 'JKO'), ('위해서', 'VV+EC'), ('도', 'JX'), ('도움', 'NNG'), ('이', 'JKS'), ('되', 'VV'), ('는', 'ETM'), ('일', 'NNG'), ('아닐까', 'VCN+EF'), ('?', 'SF'), ('지금', 'NNG'), ('까지', 'JX'), ('듣', 'VV'), ('던', 'ETM'), ('중', 'NNB'), ('가장', 'MAG'), ('신박', 'NNG'), ('한', 'XSA+ETM'), ('제안', 'NNG'), ('이', 'VCP'), ('였', 'EP'), ('습니다', 'EF'), ('.', 'SF'), ('교수', 'NNG'), ('님', 'XSN'), ('~!', 'SY'), ('서울시', 'NNP'), ('나', 'JC'), ('국토', 'NNG'), ('부', 'NNG'), ('에서', 'JKB'), ('이', 'MM'), ('영상', 'NNG'), ('을', 'JKO'), ('보', 'VV'), ('고', 'EC'), ('진지', 'XR'), ('하', 'XSA

In [45]:
tokenized_doc_revised = []
for i in range(len(tokenized_doc)):
    tmp = []
    for token in tokenized_doc[i]:
        tmp.append(token[0])
    tokenized_doc_revised.append(tmp)

In [46]:
print(tokenized_doc_revised[0])

['되', '도', '안', '한', '뉴스', '알고리즘', '에', '돈', ',', '시간', ',', '인력', '쓰', '지', '말', '고', '지하철', '시스템', '과', '연동', '해', '물류', '를', '옮길', '알고리즘', '을', '개발', '하', '는', '게', '나라', '를', '위해서', '도', '도움', '이', '되', '는', '일', '아닐까', '?', '지금', '까지', '듣', '던', '중', '가장', '신박', '한', '제안', '이', '였', '습니다', '.', '교수', '님', '~!', '서울시', '나', '국토', '부', '에서', '이', '영상', '을', '보', '고', '진지', '하', '게', '논의', '되', '면', '좋', '겠', '습니다', '.', '근데', '이게', '실현', '이', '된다면', '.', '..', '물류', ',', '택배', ',', '배송', '업계', '의', '극심', '한', '반대', '가', '예상', '됩니다', '.', '대량', '실직', '이', '니', '뭐', '니', '하', '면서', '~']


In [47]:
tokenized_doc_revised

[['되',
  '도',
  '안',
  '한',
  '뉴스',
  '알고리즘',
  '에',
  '돈',
  ',',
  '시간',
  ',',
  '인력',
  '쓰',
  '지',
  '말',
  '고',
  '지하철',
  '시스템',
  '과',
  '연동',
  '해',
  '물류',
  '를',
  '옮길',
  '알고리즘',
  '을',
  '개발',
  '하',
  '는',
  '게',
  '나라',
  '를',
  '위해서',
  '도',
  '도움',
  '이',
  '되',
  '는',
  '일',
  '아닐까',
  '?',
  '지금',
  '까지',
  '듣',
  '던',
  '중',
  '가장',
  '신박',
  '한',
  '제안',
  '이',
  '였',
  '습니다',
  '.',
  '교수',
  '님',
  '~!',
  '서울시',
  '나',
  '국토',
  '부',
  '에서',
  '이',
  '영상',
  '을',
  '보',
  '고',
  '진지',
  '하',
  '게',
  '논의',
  '되',
  '면',
  '좋',
  '겠',
  '습니다',
  '.',
  '근데',
  '이게',
  '실현',
  '이',
  '된다면',
  '.',
  '..',
  '물류',
  ',',
  '택배',
  ',',
  '배송',
  '업계',
  '의',
  '극심',
  '한',
  '반대',
  '가',
  '예상',
  '됩니다',
  '.',
  '대량',
  '실직',
  '이',
  '니',
  '뭐',
  '니',
  '하',
  '면서',
  '~'],
 ['지하철',
  '물류',
  '배송',
  '은',
  '굉장히',
  '좋',
  '은',
  '생각',
  '같',
  '습니다',
  '.',
  '<',
  'br',
  '>',
  '외곽',
  '과',
  '연결',
  '도',
  '이미',
  '되',
  '어',
  '있',
  '고',
  '<',
  'br',
  

In [48]:
dictionary = corpora.Dictionary(tokenized_doc_revised)
dictionary.filter_extremes(no_below = 10)
corpus = [dictionary.doc2bow(text) for text in tokenized_doc_revised]

In [49]:
print(corpus[0])

[(0, 4), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 2), (8, 1), (9, 2), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 2), (19, 1), (20, 1), (21, 2), (22, 1), (23, 1), (24, 3), (25, 1), (26, 1), (27, 1), (28, 2), (29, 1), (30, 1), (31, 1), (32, 2), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 2), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 2), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 3), (62, 3), (63, 1)]


In [50]:
NUM_TOPICS = 10
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(El

In [16]:
!pip install pyLDAvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 35.9 MB/s eta 0:00:00


In [51]:
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis)

/usr/local/lib/python3.8/dist-packages/pyLDAvis/_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


In [18]:
for i, topic_list in enumerate(ldamodel[corpus]):
    if i==5:
        break
    print(i,'번째 문서의 topic 비율은',topic_list)

0 번째 문서의 topic 비율은 [(2, 0.12007937), (5, 0.20910005), (6, 0.22528353), (8, 0.43821833)]
1 번째 문서의 topic 비율은 [(3, 0.1867797), (8, 0.79543847)]
2 번째 문서의 topic 비율은 [(5, 0.7057809), (6, 0.28434014)]
3 번째 문서의 topic 비율은 [(0, 0.9826867)]
4 번째 문서의 topic 비율은 [(1, 0.6954755), (4, 0.12433511), (6, 0.16496876)]


In [52]:
for i, topic_list in enumerate(ldamodel[corpus]):
    if i==5:
        break
    print(i,'번째 문서의 topic 비율은',topic_list)

0 번째 문서의 topic 비율은 [(0, 0.51301265), (1, 0.21044591), (2, 0.036994383), (6, 0.09723669), (8, 0.13621156)]
1 번째 문서의 topic 비율은 [(1, 0.6493187), (9, 0.33290023)]
2 번째 문서의 topic 비율은 [(0, 0.81498426), (5, 0.12193994), (8, 0.054431744)]
3 번째 문서의 topic 비율은 [(1, 0.12142605), (3, 0.14841501), (5, 0.29270586), (8, 0.4259125)]
4 번째 문서의 topic 비율은 [(0, 0.41156632), (4, 0.47295633), (5, 0.10025729)]
